# Wedding Constraint Satisfaction Problem Tutorial
In this project, I use [MiniZinc](https://www.minizinc.org/) to solve a wedding constraint satisfaction problem to determine the seating arrangements of guests at the wedding reception. 

In this problem, there are nGuests attending the wedding that must be seated at nTables, where each table can seat at most nSeats guests. We are also given a list Groups of size nGroups which tells us which people must be seated together. We are additionally given a list Pref of length nGuests which tells us which people a guest would prefer to sit with. We use an arbitrary, integer utility measure PrefWeight which tells us how much happiness each guest derives from sitting next to their preferred guests. The bride's mom has also given us a set of guests who all must sit at different tables, otherwise they would get too drunk and cause problems.
       
### The Constraints
We want to find a mapping of guests to tables such that:
      
* No table has more people at it than nSeats.      
* Every table must have at least Floor(nSeats/2) people at it.      
* Every person must be seated with their group.      
* Everyone in the set Trouble must be seated at different tables.    

## Measures of Utility
I investigate two measures of utility: the Utilitarian Social Welfare Function (the sum of utilities of all the individuals) and the Egalitarian Social Welfare Function (the utility of the worst off guest). 

### Step 1: Include Necessary Global Constraints

In [ ]:
%load_ext iminizinc

In [ ]:
%%minizinc

include "globals.mzn";
include "alldifferent.mzn";

### Step 2: Instantiate Variables

Create "Evaluation" variable to determine utility function being used in solution (0 = Utilitarian, 1 = Egalitarian), "PrefWeight" variable as utility measure which tells us how much happiness each guest derives from sitting next to their preferred guests, and "Trouble" set of guests who all must sit at different tables.

Create "Groups" array to represent groups of wedding guests, "Guests" set to represent each wedding guest as a unique integer, "Tables" set to determine the number of tables available for guests to sit at, and "Seats" set to determine the number of seats available for guests to sit at.

Create "AtTable" array which determines the set of guests seated at each table, "GuestAt" array which determines the table that each group is seated at, and "Pref" array which determines the set of people each guest prefers to sit with.

In [ ]:
%%minizinc -m bind

int: Evaluation; 
int: PrefWeight;
set of int: Trouble;

int: nGroups;
set of int: GROUPS = 1..nGroups;
array[GROUPS] of set of int: Groups;

int: nGuests;
set of int: GUESTS = 1..nGuests; 

int: nTables;  
set of int: TABLES = 1..nTables; 

int: nSeats;
set of int: SEATS = 1..nSeats;

array[TABLES] of var set of GUESTS: AtTable;

array[GUESTS] of var TABLES: GuestAt;

array[GUESTS] of set of int: Pref;

### Step 3: Define the Constraints

First, to ensure that each guest index position in the GuestAt array corresponds to the correct table index position in the AtTable, I use the **int_set_channel** predicate such that (GuestAt[i] = j) ↔ (i in AtTable[j]). 

In [ ]:
%%minizinc

constraint int_set_channel(GuestAt, AtTable);

Next, so that each guest is uniquely assigned to only one table, I use the **all_disjoint** constraint.

In [ ]:
%%minizinc

constraint all_disjoint(AtTable);

**Constraint 1: No table has more people at it than nSeats.**

In [ ]:
%%minizinc

constraint forall (k in TABLES) (card(AtTable[k]) <= nSeats);

**Constraint 2: Every table must have at least Floor(nSeats/2) people at it.**

In [ ]:
%%minizinc

constraint forall (k in TABLES) (card(AtTable[k]) >= floor(nSeats/2));

**Constraint 3: Every person must be seated with their group.**

In [ ]:
%%minizinc

constraint forall(g in GROUPS)(all_equal([GuestAt[p] | p in Groups[g]]));

**Constraint 4: Everyone in the set Trouble must be seated at different tables.**

In [ ]:
%%minizinc

constraint alldifferent(t in Trouble)(AtTable[t]);

### Step 4: Symmetry Breaking

In order to prevent symmetry variants of my solution, I use a modelling technique called *symmetry breaking.* This involves adding constraints to the model that rule out all symmetric variants of a (partial) assignment to the variables except one.
      
To do this, I impose a constraint that forces the model to order table seating by group order in the solution. This will remove the possibility of any symmetry variants and ensure that there is only one solution to this problem.

In [ ]:
%%minizinc

constraint forall (g in GROUPS) (Groups[g] subset AtTable[g]);

### Step 5: Solve 

Here, I investigate the utiliarian and egalitarian social welfare functions to maximize the happiness of all the guests at the wedding. The **Utilitarian Social Welfare Function** measures the sum of utilities of all the guests. The **Egalitarian Social Welfare Function** measures the utlity of the worst off guest. 
     
The solution seeks to maximize the utility measured by one of these functions.

In [ ]:
%%minizinc

var int: util = sum (i in GUESTS) (card(Pref[i] intersect AtTable[GuestAt[i]]) * PrefWeight);

var int: egal = min (i in GUESTS) (card(Pref[i] intersect AtTable[GuestAt[i]]) * PrefWeight);

solve maximize (if Evaluation == 0 then util else egal endif);

### Output

In [ ]:
%%minizinc

output ["Table: Guests: " ++ "\n"]; 
output ["   " ++ show(i) ++ " : " ++ show(AtTable[i]) ++ "\n" | i in TABLES ];
output ["Utilitarian Happiness: " ++ show(util) ++ "\nEgalitarian Happiness: " ++ show(egal) ];

### Step 6: Run Solution

First, I create a .dzn file for the problem data which specifies the values of each variable. The example .dzn file below shows problem data that will be scored using a utilitarian welfare function.

Next, I run my .dzn file and .mzn files and return an output file of the solution. The output of the solution for the above problem data is shown below. 

From the example output file above, we can see there is one solution for the problem that has a score of 69 for Utilitarian happiness and 0 for Egalitarian happiness. 